# Assignment 5

In [1]:
import numpy as np
import pandas as pd 
import scipy as sci
import math 
import matplotlib as mp
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

from itertools import chain
from numpy import pi
from scipy import interpolate
from scipy.signal import freqz, welch, filtfilt, ellip
from scipy.interpolate import interp1d, CubicSpline

from matplotlib.dates import DateFormatter, MinuteLocator, HourLocator, MonthLocator
from matplotlib.ticker import FormatStrFormatter, StrMethodFormatter
from matplotlib.ticker import FixedFormatter

from textwrap import wrap

%matplotlib inline
%config InlineBackend.figure_format = 'pdf'

### Import data

In [2]:
AS = pd.read_csv('/Users/Kev/Documents/Uvic/Python/PHYS 411 - Time Series Analysis/Data Sets/AllStations_temperature_h_2017.dat', 
                 sep='\s+', header=[0, 1])

DC = pd.read_csv('/Users/Kev/Documents/Uvic/Python/PHYS 411 - Time Series Analysis/Data Sets/DeepCove_temperature.dat', 
                 header=2)

JB = pd.read_csv('/Users/Kev/Documents/Uvic/Python/PHYS 411 - Time Series Analysis/Data Sets/JamesBay_temperature.dat', 
                 header=2)

### Clean up data

In [3]:
# Clean up minute resolution data 
# -------------------------------

def DateInsert(Data):
    date = pd.date_range(start='2011-12-31 17:00:00.000000', 
                     freq='min', periods = len(Data))
    # Insert dates into D2 dataframe
    Data.insert(loc=0, column='Time', value=date)
    # Rename the columns
    D1 = Data.rename(index=str, columns={Data.columns[1]: "Temperature"})
    # Set index
    DM = D1.set_index('Time')
    return DM

DataM = [DC, JB]
DataMLabels = ['Deep Cove', 'James Bay']

# Insert dates
DataMIns = [DateInsert(DataM[i]) for i in range(0, len(DataM))]

In [4]:
# Clean up hour resolution data
# -----------------------------

# Convert times from MatLab time to Python Time
AS['Time'] = AS['NaN', 'NaN'].apply(lambda matlab_datenum: 
                             dt.datetime.fromordinal(int(matlab_datenum)) 
                             + dt.timedelta(days=matlab_datenum%1)
                             - dt.timedelta(days = 366)) 

# Rename the columns
AS2 = AS.rename(index=str, columns={"NaN": "MatLab Time"})

# Set time as index column
AS3 = AS2.set_index('Time')

# Get the longitudes correct 
for i in range(1, np.shape(AS3)[1]):
    New_name = float(list(AS3)[i][1]) - 360
    AS3 = AS3.rename(columns={list(AS)[i][1]: str(format(New_name, '.4f'))})

AS4 = AS3.drop('MatLab Time', axis=1, level=1)
    
AS = AS4['48.4623']['-123.3090']

### Select data time

In [5]:
# Select dates
DCs = DataMIns[0].loc['2015-06-01 00:00':'2015-09-30 23:59']['Temperature']
JBs = DataMIns[1].loc['2015-06-01 00:00':'2015-09-30 23:59']['Temperature']
DCw = DataMIns[0].loc['2014-11-01 00:00':'2015-02-28 23:59']['Temperature']
JBw = DataMIns[1].loc['2014-11-01 00:00':'2015-02-28 23:59']['Temperature']

US = AS.loc['2016-01-01 00:00':'2016-12-30 23:59']
USm = AS.loc['2016-05-01 00:00':'2016-05-30 23:59']

## Question 1: Power within a frequency band 

In [6]:
# Plot out the time series 
# ========================

fig, (DCs_Plot, JBs_Plot, DCw_Plot, JBw_Plot) = plt.subplots(4, 1, figsize=(12, 16))

DCs_Plot.plot(DCs, linewidth=1)
DCw_Plot.plot(DCw, linewidth=1)
JBs_Plot.plot(JBs, linewidth=1)
JBw_Plot.plot(JBw, linewidth=1)

DCs_Plot.xaxis.set_major_locator(mp.dates.MonthLocator(bymonthday = (1)))
DCw_Plot.xaxis.set_major_locator(mp.dates.MonthLocator(bymonthday = (1)))
JBs_Plot.xaxis.set_major_locator(mp.dates.MonthLocator(bymonthday = (1)))
JBw_Plot.xaxis.set_major_locator(mp.dates.MonthLocator(bymonthday = (1)))

DCs_Plot.xaxis.set_major_formatter(DateFormatter('%b-%Y'))
DCw_Plot.xaxis.set_major_formatter(DateFormatter('%b-%Y'))
JBs_Plot.xaxis.set_major_formatter(DateFormatter('%b-%Y'))
JBw_Plot.xaxis.set_major_formatter(DateFormatter('%b-%Y'))

DCs_Plot.set_title("Minute resolution measuerments of temperature at Deep Cove from 2015-06-01 to 2015-09-30")
DCw_Plot.set_title("Minute resolution measuerments of temperature at Deep Cove from 2014-11-01 to 2015-02-28")
JBs_Plot.set_title("Minute resolution measuerments of temperature at James Bay from 2015-06-01 to 2015-09-30")
JBw_Plot.set_title("Minute resolution measuerments of temperature at James Bay from 2014-11-01 to 2015-02-28")

DCs_Plot.set_xlabel("Time")
DCw_Plot.set_xlabel("Time")
JBs_Plot.set_xlabel("Time")
JBw_Plot.set_xlabel("Time")

DCs_Plot.set_ylabel(r"Temperature [$^\circ$ C]")
DCw_Plot.set_ylabel(r"Temperature [$^\circ$ C]")
JBs_Plot.set_ylabel(r"Temperature [$^\circ$ C]")
JBw_Plot.set_ylabel(r"Temperature [$^\circ$ C]")

fig.tight_layout()
plt.show()

<Figure size 864x1152 with 4 Axes>

In [7]:
# Calculate the power spectra
# ===========================

# De-Nan
DCs_DN = DCs[~np.isnan(DCs)]
JBs_DN = JBs[~np.isnan(JBs)]
DCw_DN = DCw[~np.isnan(DCw)]
JBw_DN = JBw[~np.isnan(JBw)]
FSm = 1/60

# Lengths 
[DCs_DN_L, JBs_DN_L, DCw_DN_L, JBw_DN_L] = np.array([len(DCs_DN), len(JBs_DN), len(DCw_DN), len(JBw_DN)]) / (2**2)

# Now calculate
FDCs, GDCs = welch(DCs_DN, FSm, nperseg=DCs_DN_L,\
            window=sci.signal.windows.hann(int(DCs_DN_L)),\
            noverlap = DCs_DN_L/2, nfft = DCs_DN_L, detrend=False,\
            return_onesided=True, scaling = 'spectrum')

FDCw, GDCw = welch(DCw_DN, FSm, nperseg=JBs_DN_L,\
            window=sci.signal.windows.hann(int(JBs_DN_L)),\
            noverlap = JBs_DN_L/2, nfft = JBs_DN_L, detrend=False,\
            return_onesided=True, scaling = 'spectrum')

FJBs, GJBs = welch(JBs_DN, FSm, nperseg=DCw_DN_L,\
            window=sci.signal.windows.hann(int(DCw_DN_L)),\
            noverlap = DCw_DN_L/2, nfft = DCw_DN_L, detrend=False,\
            return_onesided=True, scaling = 'spectrum')

FJBw, GJBw = welch(JBw_DN, FSm, nperseg=JBw_DN_L,\
            window=sci.signal.windows.hann(int(JBw_DN_L)),\
            noverlap = JBw_DN_L/2, nfft = JBw_DN_L, detrend=False,\
            return_onesided=True, scaling = 'spectrum')

In [8]:
# Plot the power spectra
# ======================
fig, (DCs_PS, JBs_PS, DCw_PS, JBw_PS) = plt.subplots(4, 1, figsize=(12, 16))
# plt.loglog(Fx/(2*pi), Gx, label = r'PDS of $x(t)$')
DCs_PS.loglog(FDCs/(2*pi), GDCs, label = r'PDS of $x_f(t)$', linewidth=1)
JBs_PS.loglog(FJBs/(2*pi), GJBs, label = r'PDS of $x_f(t)$', linewidth=1)
DCw_PS.loglog(FDCw/(2*pi), GDCw, label = r'PDS of $x_f(t)$', linewidth=1)
JBw_PS.loglog(FJBw/(2*pi), GJBw, label = r'PDS of $x_f(t)$', linewidth=1)

DCs_PS.set_title("Power spectrum of minute resolution measuerments of temperature at Deep Cove from 2015-06-01 to 2015-09-30")
DCw_PS.set_title("Power spectrum of minute resolution measuerments of temperature at Deep Cove from 2014-11-01 to 2015-02-28")
JBs_PS.set_title("Power spectrum of minute resolution measuerments of temperature at James Bay from 2015-06-01 to 2015-09-30")
JBw_PS.set_title("Power spectrum of minute resolution measuerments of temperature at James Bay from 2014-11-01 to 2015-02-28")

DCs_PS.set_xlabel("Frequency [Hz]")
DCw_PS.set_xlabel("Frequency [Hz]")
JBs_PS.set_xlabel("Frequency [Hz]")
JBw_PS.set_xlabel("Frequency [Hz]")

DCs_PS.set_ylabel(r'$G_{xx}(f)$ [$\frac{V^2}{Hz}$]')
DCw_PS.set_ylabel(r'$G_{xx}(f)$ [$\frac{V^2}{Hz}$]')
JBs_PS.set_ylabel(r'$G_{xx}(f)$ [$\frac{V^2}{Hz}$]')
JBw_PS.set_ylabel(r'$G_{xx}(f)$ [$\frac{V^2}{Hz}$]')

fig.tight_layout()
plt.show()

<Figure size 864x1152 with 4 Axes>

#### a) Elliptic filters

In [9]:
# Filter functions
# =================

# High/Low-pass filters 
def Ellip(Cut, rp, rs, fs, order, btype):
    fn = fs/2
    Ws = Cut / fn
    b, a = ellip(order, rp, rs, Ws, btype, output='ba')
    return b, a

def EllipF(x, Cut, rp, rs, fs, order, btype):
    b, a = Ellip(Cut, rp, rs, fs, order, btype)
    y = filtfilt(b, a, x)
    return y

def GraphEllip(Cut, rp, rs, fs, order, btype, worN=2*10**5):
    b, a = Ellip(Cut, rp, rs, fs, order, btype)
    w, h = freqz(b, a, worN)
    return b, a, w, h

# Bandpass filters 
def EllipBP(CutL, CutH, rp, rs, fs, order, btype):
    fn = fs/2
    low = CutL / fn
    high = CutH / fn
    b, a = ellip(order, rp, rs, [low, high], btype, output='ba')
    return b, a

def EllipFBP(x, CutL, CutH, rp, rs, fs, order, btype):
    b, a = EllipBP(CutL, CutH, rp, rs, fs, order, btype)
    y = filtfilt(b, a, x)
    return y

def GraphEllipBP(CutL, CutH, rp, rs, fs, order, btype, worN=2*10**5):
    b, a = EllipBP(CutL, CutH, rp, rs, fs, order, btype)
    w, h = freqz(b, a, worN)
    return b, a, w, h

In [41]:
# Ellipse filter settings
[DCs_CutH, DCs_rpH, DCs_rsH, DCs_orderH, DCs_btypeH] = [0.75*10**(-5)/(2*pi), 25, 150, 2, 'highpass']
[DCs_CutL, DCs_rpL, DCs_rsL, DCs_orderL, DCs_btypeL] = [1.75*10**(-5)/(2*pi), 25, 150, 2, 'lowpass']
[DCs_CutBL, DCs_CutBH, DCs_rpB, DCs_rsB, DCs_orderB, DCs_btypeB] = [0.825*10**(-5)/(2*pi), 1.63*10**(-5)/(2*pi), 12, 150, 2, 'bandpass']

[DCw_CutH, DCw_rpH, DCw_rsH, DCw_orderH, DCw_btypeH] = [0.78*10**(-5)/(2*pi), 25, 150, 2, 'highpass']
[DCw_CutL, DCw_rpL, DCw_rsL, DCw_orderL, DCw_btypeL] = [1.75*10**(-5)/(2*pi), 25, 150, 2, 'lowpass']
[DCw_CutBL, DCw_CutBH, DCw_rpB, DCw_rsB, DCw_orderB, DCw_btypeB] = [0.825*10**(-5)/(2*pi), 1.63*10**(-5)/(2*pi), 12, 150, 2, 'bandpass']

[JBs_CutH, JBs_rpH, JBs_rsH, JBs_orderH, JBs_btypeH] = [0.77*10**(-5)/(2*pi), 25, 150, 2, 'highpass']
[JBs_CutL, JBs_rpL, JBs_rsL, JBs_orderL, JBs_btypeL] = [1.75*10**(-5)/(2*pi), 25, 150, 2, 'lowpass']
[JBs_CutBL, JBs_CutBH, JBs_rpB, JBs_rsB, JBs_orderB, JBs_btypeB] = [0.825*10**(-5)/(2*pi), 1.63*10**(-5)/(2*pi), 12, 150, 2, 'bandpass']

[JBw_CutH, JBw_rpH, JBw_rsH, JBw_orderH, JBw_btypeH] = [0.77*10**(-5)/(2*pi), 25, 150, 2, 'highpass']
[JBw_CutL, JBw_rpL, JBw_rsL, JBw_orderL, JBw_btypeL] = [1.75*10**(-5)/(2*pi), 25, 150, 2, 'lowpass']
[JBw_CutBL, JBw_CutBH, JBw_rpB, JBw_rsB, JBw_orderB, JBw_btypeB] = [0.825*10**(-5)/(2*pi), 1.63*10**(-5)/(2*pi), 12, 150, 2, 'bandpass']

In [42]:
DCs_bH, DCs_aH, DCs_wH, DCs_hH = GraphEllip(Cut=DCs_CutH, rp=DCs_rpH, rs=DCs_rsH, fs=FSm, order=DCs_orderH, btype=DCs_btypeH)
DCs_bL, DCs_aL, DCs_wL, DCs_hL = GraphEllip(Cut=DCs_CutL, rp=DCs_rpL, rs=DCs_rsL, fs=FSm, order=DCs_orderL, btype=DCs_btypeL)
DCs_bB, DCs_aB, DCs_wB, DCs_hB = GraphEllipBP(CutL=DCs_CutBL, CutH=DCs_CutBH, rp=DCs_rpB, rs=DCs_rsB, fs=FSm, order=DCs_orderB, btype=DCs_btypeB)

DCw_bH, DCw_aH, DCw_wH, DCw_hH = GraphEllip(Cut=DCw_CutH, rp=DCw_rpH, rs=DCw_rsH, fs=FSm, order=DCw_orderH, btype=DCw_btypeH)
DCw_bL, DCw_aL, DCw_wL, DCw_hL = GraphEllip(Cut=DCw_CutL, rp=DCw_rpL, rs=DCw_rsL, fs=FSm, order=DCw_orderL, btype=DCw_btypeL)
DCw_bB, DCw_aB, DCw_wB, DCw_hB = GraphEllipBP(CutL=DCw_CutBL, CutH=DCw_CutBH, rp=DCw_rpB, rs=DCw_rsB, fs=FSm, order=DCw_orderB, btype=DCw_btypeB)

JBs_bH, JBs_aH, JBs_wH, JBs_hH = GraphEllip(Cut=JBs_CutH, rp=JBs_rpH, rs=JBs_rsH, fs=FSm, order=JBs_orderH, btype=JBs_btypeH)
JBs_bL, JBs_aL, JBs_wL, JBs_hL = GraphEllip(Cut=JBs_CutL, rp=JBs_rpL, rs=JBs_rsL, fs=FSm, order=JBs_orderL, btype=JBs_btypeL)
JBs_bB, JBs_aB, JBs_wB, JBs_hB = GraphEllipBP(CutL=JBs_CutBL, CutH=JBs_CutBH, rp=JBs_rpB, rs=JBs_rsB, fs=FSm, order=JBs_orderB, btype=JBs_btypeB)

JBw_bH, JBw_aH, JBw_wH, JBw_hH = GraphEllip(Cut=JBw_CutH, rp=JBw_rpH, rs=JBw_rsH, fs=FSm, order=JBw_orderH, btype=JBw_btypeH)
JBw_bL, JBw_aL, JBw_wL, JBw_hL = GraphEllip(Cut=JBw_CutL, rp=JBw_rpL, rs=JBw_rsL, fs=FSm, order=JBw_orderL, btype=JBw_btypeL)
JBw_bB, JBw_aB, JBw_wB, JBw_hB = GraphEllipBP(CutL=JBw_CutBL, CutH=JBw_CutBH, rp=JBw_rpB, rs=JBw_rsB, fs=FSm, order=JBw_orderB, btype=JBw_btypeB)

In [43]:
# Plot the filter
fig, (DCs_ellip_plot, DCw_ellip_plot, JBs_ellip_plot, JBw_ellip_plot) = plt.subplots(4, 1, figsize=(10, 16))

DCs_ellip_plot.loglog(FDCs/(2*pi), GDCs, label = r'PDS of Deep Cove summer data', linewidth=1)
DCs_ellip_plot.loglog(FSm/(2*pi) * DCs_wH, abs(DCs_hH), label="Type: {0}, Order: {1}, Cut-off: {2:.1e} Hz".format(DCs_btypeH, DCs_orderH, DCs_CutH))
DCs_ellip_plot.loglog(FSm/(2*pi) * DCs_wL, abs(DCs_hL), label="Type: {0}, Order: {1}, Cut-off: {2:.1e} Hz".format(DCs_btypeL, DCs_orderL, DCs_CutL))
DCs_ellip_plot.loglog(FSm/(2*pi) * DCs_wB, abs(DCs_hB), label="Type: {0}, Order: {1}, Cut-off: {2:.1e} Hz and {3:.1e} Hz".format(DCs_btypeB, DCs_orderB, DCs_CutBL, DCs_CutBH))
DCs_ellip_plot.set_title('Elliptic filter frequency responce')
DCs_ellip_plot.set_xlabel(r'Frequency ($Hz$)')
DCs_ellip_plot.set_ylabel(r'$H(\omega)$')
DCs_ellip_plot.grid(True)
DCs_ellip_plot.legend(loc=3)

DCw_ellip_plot.loglog(FDCw/(2*pi), GDCw, label = r'PDS of Deep Cove winter data', linewidth=1)
DCw_ellip_plot.loglog(FSm/(2*pi) * DCw_wH, abs(DCw_hH), label="Type: {0}, Order: {1}, Cut-off: {2:.1e} Hz".format(DCw_btypeH, DCw_orderH, DCw_CutH))
DCw_ellip_plot.loglog(FSm/(2*pi) * DCw_wL, abs(DCw_hL), label="Type: {0}, Order: {1}, Cut-off: {2:.1e} Hz".format(DCw_btypeL, DCw_orderL, DCw_CutL))
DCw_ellip_plot.loglog(FSm/(2*pi) * DCw_wB, abs(DCw_hB), label="Type: {0}, Order: {1}, Cut-off: {2:.1e} Hz and {3:.1e} Hz".format(DCw_btypeB, DCw_orderB, DCw_CutBL, DCw_CutBH))
DCw_ellip_plot.set_title('Elliptic filter frequency responce')
DCw_ellip_plot.set_xlabel(r'Frequency ($Hz$)')
DCw_ellip_plot.set_ylabel(r'$H(\omega)$')
DCw_ellip_plot.grid(True)
DCw_ellip_plot.legend(loc=3)

JBs_ellip_plot.loglog(FJBs/(2*pi), GJBs, label = r'PDS of James Bay summer data', linewidth=1)
JBs_ellip_plot.loglog(FSm/(2*pi) * JBs_wH, abs(JBs_hH), label="Type: {0}, Order: {1}, Cut-off: {2:.1e} Hz".format(JBs_btypeH, JBs_orderH, JBs_CutH))
JBs_ellip_plot.loglog(FSm/(2*pi) * JBs_wL, abs(JBs_hL), label="Type: {0}, Order: {1}, Cut-off: {2:.1e} Hz".format(JBs_btypeL, JBs_orderL, JBs_CutL))
JBs_ellip_plot.loglog(FSm/(2*pi) * JBs_wB, abs(JBs_hB), label="Type: {0}, Order: {1}, Cut-off: {2:.1e} Hz and {3:.1e} Hz".format(JBs_btypeB, JBs_orderB, JBs_CutBL, JBs_CutBH))
JBs_ellip_plot.set_title('Elliptic filter frequency responce')
JBs_ellip_plot.set_xlabel(r'Frequency ($Hz$)')
JBs_ellip_plot.set_ylabel(r'$H(\omega)$')
JBs_ellip_plot.grid(True)
JBs_ellip_plot.legend(loc=3)

JBw_ellip_plot.loglog(FJBw/(2*pi), GJBw, label = r'PDS of James bay winter data', linewidth=1)
JBw_ellip_plot.loglog(FSm/(2*pi) * JBw_wH, abs(JBw_hH), label="Type: {0}, Order: {1}, Cut-off: {2:.1e} Hz".format(JBw_btypeH, JBw_orderH, JBw_CutH))
JBw_ellip_plot.loglog(FSm/(2*pi) * JBw_wL, abs(JBw_hL), label="Type: {0}, Order: {1}, Cut-off: {2:.1e} Hz".format(JBw_btypeL, JBw_orderL, JBw_CutL))
JBw_ellip_plot.loglog(FSm/(2*pi) * JBw_wB, abs(JBw_hB), label="Type: {0}, Order: {1}, Cut-off: {2:.1e} Hz and {3:.1e} Hz".format(JBw_btypeB, JBw_orderB, JBw_CutBL, JBw_CutBH))
JBw_ellip_plot.set_title('Elliptic filter frequency responce')
JBw_ellip_plot.set_xlabel(r'Frequency ($Hz$)')
JBw_ellip_plot.set_ylabel(r'$H(\omega)$')
JBw_ellip_plot.grid(True)
JBw_ellip_plot.legend(loc=3)

fig.tight_layout()
plt.show()

<Figure size 720x1152 with 4 Axes>

In [44]:
# Filter the time series 
DCsFH = EllipF(DCs, Cut=DCs_CutH, rp=DCs_rpH, rs=DCs_rsH, fs=FSm, order=DCs_orderH, btype=DCs_btypeH)
DCsFL = EllipF(DCs, Cut=DCs_CutL, rp=DCs_rpL, rs=DCs_rsL, fs=FSm, order=DCs_orderL, btype=DCs_btypeL)
DCsFHL = EllipF(DCsFH, Cut=DCs_CutL, rp=DCs_rpL, rs=DCs_rsL, fs=FSm, order=DCs_orderL, btype=DCs_btypeL)
DCsFB = EllipFBP(DCs, CutL=DCs_CutBL, CutH=DCs_CutBH, rp=DCs_rpB, rs=DCs_rsB, fs=FSm, order=DCs_orderB, btype=DCs_btypeB)

DCwFH = EllipF(DCw[~np.isnan(DCw)], Cut=DCw_CutH, rp=DCw_rpH, rs=DCw_rsH, fs=FSm, order=DCw_orderH, btype=DCw_btypeH)
DCwFL = EllipF(DCw[~np.isnan(DCw)], Cut=DCw_CutL, rp=DCw_rpL, rs=DCw_rsL, fs=FSm, order=DCw_orderL, btype=DCw_btypeL)
DCwFHL = EllipF(DCwFH, Cut=DCw_CutL, rp=DCw_rpL, rs=DCw_rsL, fs=FSm, order=DCw_orderL, btype=DCw_btypeL)
DCwFB = EllipFBP(DCw[~np.isnan(DCw)], CutL=DCw_CutBL, CutH=DCw_CutBH, rp=DCw_rpB, rs=DCw_rsB, fs=FSm, order=DCw_orderB, btype=DCw_btypeB)

JBsFH = EllipF(JBs[~np.isnan(JBs)], Cut=JBs_CutH, rp=JBs_rpH, rs=JBs_rsH, fs=FSm, order=JBs_orderH, btype=JBs_btypeH)
JBsFL = EllipF(JBs[~np.isnan(JBs)], Cut=JBs_CutL, rp=JBs_rpL, rs=JBs_rsL, fs=FSm, order=JBs_orderL, btype=JBs_btypeL)
JBsFHL = EllipF(JBsFH, Cut=JBs_CutL, rp=JBs_rpL, rs=JBs_rsL, fs=FSm, order=JBs_orderL, btype=JBs_btypeL)
JBsFB = EllipFBP(JBs[~np.isnan(JBs)], CutL=JBs_CutBL, CutH=JBs_CutBH, rp=JBs_rpB, rs=JBs_rsB, fs=FSm, order=JBs_orderB, btype=JBs_btypeB)

JBwFH = EllipF(JBw[~np.isnan(JBw)], Cut=JBw_CutH, rp=JBw_rpH, rs=JBw_rsH, fs=FSm, order=JBw_orderH, btype=JBw_btypeH)
JBwFL = EllipF(JBw[~np.isnan(JBw)], Cut=JBw_CutL, rp=JBw_rpL, rs=JBw_rsL, fs=FSm, order=DCs_orderL, btype=JBw_btypeL)
JBwFHL = EllipF(JBwFH, Cut=JBw_CutL, rp=JBw_rpL, rs=JBw_rsL, fs=FSm, order=JBw_orderL, btype=JBw_btypeL)
JBwFB = EllipFBP(JBw[~np.isnan(JBw)], CutL=JBw_CutBL, CutH=JBw_CutBH, rp=JBw_rpB, rs=JBw_rsB, fs=FSm, order=JBw_orderB, btype=JBw_btypeB)


In [45]:
# Plot for DCs
dr_DCs = pd.date_range(DCs.index[0], DCs.index[-1], freq='m')
t_DCs = DCs.index.values

fig, (DCs_Plot2, DCs_FH, DCs_FL, DCs_FHL, DCs_FB) = plt.subplots(5, 1, figsize=(10, 16))
DCs_Plot2.plot(DCs, linewidth=1)
DCs_FL.plot(t_DCs, DCsFH)
DCs_FH.plot(t_DCs, DCsFL)
DCs_FHL.plot(t_DCs, DCsFHL)
DCs_FB.plot(t_DCs, DCsFB)

DCs_Plot2.xaxis.set_major_locator(mp.dates.MonthLocator(bymonthday = (1)))
DCs_FL.xaxis.set_major_locator(mp.dates.MonthLocator(bymonthday = (1)))
DCs_FH.xaxis.set_major_locator(mp.dates.MonthLocator(bymonthday = (1)))
DCs_FHL.xaxis.set_major_locator(mp.dates.MonthLocator(bymonthday = (1)))
DCs_FB.xaxis.set_major_locator(mp.dates.MonthLocator(bymonthday = (1)))

DCs_Plot2.xaxis.set_major_formatter(DateFormatter('%b-%Y'))
DCs_FL.xaxis.set_major_formatter(DateFormatter('%b-%Y'))
DCs_FH.xaxis.set_major_formatter(DateFormatter('%b-%Y'))
DCs_FHL.xaxis.set_major_formatter(DateFormatter('%b-%Y'))
DCs_FB.xaxis.set_major_formatter(DateFormatter('%b-%Y'))

DCs_Plot2.set_title(r'Temperature values from Deep Cove at hour resolution from June 2015 to Sept 2015')

DCs_FL_title = r'Diurnal temperature values from Deep Cove at hour resolution from June 2015 to Sept 2015 extracted using a lowpass elliptical filter'
DCs_FH_title = r'Diurnal temperature values from Deep Cove at hour resolution from June 2015 to Sept 2015 extracted using a highpass elliptical filter'
DCs_FHL_title = r'Diurnal temperature values from Deep Cove at hour resolution from June 2015 to Sept 2015 extracted using a lowpass then a highpass elliptical filter'
DCs_FB_title = r'Diurnal temperature values from Deep Cove at hour resolution from June 2015 to Sept 2015 extracted using a bandpass elliptical filter'
DCs_FL.set_title("\n".join(wrap(DCs_FL_title, 90)))
DCs_FH.set_title("\n".join(wrap(DCs_FH_title, 90)))
DCs_FHL.set_title("\n".join(wrap(DCs_FHL_title, 90)))
DCs_FB.set_title("\n".join(wrap(DCs_FB_title, 90)))

DCs_Plot2.set_xlabel(r'Dates')
DCs_FL.set_xlabel(r'Dates')
DCs_FH.set_xlabel(r'Dates')
DCs_FHL.set_xlabel(r'Dates')
DCs_FB.set_xlabel(r'Dates')

DCs_Plot2.set_ylabel(r'Temperature [$^\circ$ C]')
DCs_FL.set_ylabel(r'Temperature [$^\circ$ C]')
DCs_FH.set_ylabel(r'Temperature [$^\circ$ C]')
DCs_FHL.set_ylabel(r'Temperature [$^\circ$ C]')
DCs_FB.set_ylabel(r'Temperature [$^\circ$ C]')

fig.tight_layout()
plt.show()

# Plot for DCw
dr_DCw = pd.date_range(DCw.index[0], DCw.index[-1], freq='m')
t_DCw = DCw.index.values

fig, (DCw_Plot2, DCw_FH, DCw_FL, DCw_FHL, DCw_FB) = plt.subplots(5, 1, figsize=(10, 16))
DCw_Plot2.plot(DCw, linewidth=1)
DCw_FL.plot(t_DCw[:len(DCwFH)], DCwFH)
DCw_FH.plot(t_DCw[:len(DCwFL)], DCwFL)
DCw_FHL.plot(t_DCw[:len(DCwFHL)], DCwFHL)
DCw_FB.plot(t_DCw[:len(DCwFB)], DCwFB)

DCw_Plot2.xaxis.set_major_locator(mp.dates.MonthLocator(bymonthday = (1)))
DCw_FL.xaxis.set_major_locator(mp.dates.MonthLocator(bymonthday = (1)))
DCw_FH.xaxis.set_major_locator(mp.dates.MonthLocator(bymonthday = (1)))
DCw_FHL.xaxis.set_major_locator(mp.dates.MonthLocator(bymonthday = (1)))
DCw_FB.xaxis.set_major_locator(mp.dates.MonthLocator(bymonthday = (1)))

DCw_Plot2.xaxis.set_major_formatter(DateFormatter('%b-%Y'))
DCw_FL.xaxis.set_major_formatter(DateFormatter('%b-%Y'))
DCw_FH.xaxis.set_major_formatter(DateFormatter('%b-%Y'))
DCw_FHL.xaxis.set_major_formatter(DateFormatter('%b-%Y'))
DCw_FB.xaxis.set_major_formatter(DateFormatter('%b-%Y'))

DCw_Plot2.set_title(r'Temperature values from Deep Cove at hour resolution from June 2015 to Sept 2015')

DCw_FL_title = r'Diurnal temperature values from Deep Cove at hour resolution from Nov 2014 to Feb 2015 extracted using a lowpass elliptical filter'
DCw_FH_title = r'Diurnal temperature values from Deep Cove at hour resolution from Nov 2014 to Feb 2015 extracted using a highpass elliptical filter'
DCw_FHL_title = r'Diurnal temperature values from Deep Cove at hour resolution from Nov 2014 to Feb 2015 extracted using a lowpass then a highpass elliptical filter'
DCw_FB_title = r'Diurnal temperature values from Deep Cove at hour resolution from Nov 2014 to Feb 2015 extracted using a bandpass elliptical filter'
DCw_FL.set_title("\n".join(wrap(DCw_FL_title, 90)))
DCw_FH.set_title("\n".join(wrap(DCw_FH_title, 90)))
DCw_FHL.set_title("\n".join(wrap(DCw_FHL_title, 90)))
DCw_FB.set_title("\n".join(wrap(DCw_FB_title, 90)))

DCw_Plot2.set_xlabel(r'Dates')
DCw_FL.set_xlabel(r'Dates')
DCw_FH.set_xlabel(r'Dates')
DCw_FHL.set_xlabel(r'Dates')
DCw_FB.set_xlabel(r'Dates')

DCw_Plot2.set_ylabel(r'Temperature [$^\circ$ C]')
DCw_FL.set_ylabel(r'Temperature [$^\circ$ C]')
DCw_FH.set_ylabel(r'Temperature [$^\circ$ C]')
DCw_FHL.set_ylabel(r'Temperature [$^\circ$ C]')
DCw_FB.set_ylabel(r'Temperature [$^\circ$ C]')

fig.tight_layout()
plt.show()

# Plot for JBs
dr_JBs = pd.date_range(JBs.index[0], JBs.index[-1], freq='m')
t_JBs = JBs.index.values

fig, (JBs_Plot2, JBs_FH, JBs_FL, JBs_FHL, JBs_FB) = plt.subplots(5, 1, figsize=(10, 16))
JBs_Plot2.plot(JBs, linewidth=1)
JBs_FL.plot(t_JBs[:len(JBsFH)], JBsFH)
JBs_FH.plot(t_JBs[:len(JBsFL)], JBsFL)
JBs_FHL.plot(t_JBs[:len(JBsFHL)], JBsFHL)
JBs_FB.plot(t_JBs[:len(JBsFB)], JBsFB)

JBs_Plot2.xaxis.set_major_locator(mp.dates.MonthLocator(bymonthday = (1)))
JBs_FL.xaxis.set_major_locator(mp.dates.MonthLocator(bymonthday = (1)))
JBs_FH.xaxis.set_major_locator(mp.dates.MonthLocator(bymonthday = (1)))
JBs_FHL.xaxis.set_major_locator(mp.dates.MonthLocator(bymonthday = (1)))
JBs_FB.xaxis.set_major_locator(mp.dates.MonthLocator(bymonthday = (1)))

JBs_Plot2.xaxis.set_major_formatter(DateFormatter('%b-%Y'))
JBs_FL.xaxis.set_major_formatter(DateFormatter('%b-%Y'))
JBs_FH.xaxis.set_major_formatter(DateFormatter('%b-%Y'))
JBs_FHL.xaxis.set_major_formatter(DateFormatter('%b-%Y'))
JBs_FB.xaxis.set_major_formatter(DateFormatter('%b-%Y'))

JBs_Plot2.set_title(r'Temperature values from James Bay at hour resolution from June 2015 to Sept 2015')

JBs_FL_title = r'Diurnal temperature values from James Bay at hour resolution from June 2015 to Sept 2015 extracted using a lowpass elliptical filter'
JBs_FH_title = r'Diurnal temperature values from James Bay at hour resolution from June 2015 to Sept 2015 extracted using a highpass elliptical filter'
JBs_FHL_title = r'Diurnal temperature values from James Bay at hour resolution from June 2015 to Sept 2015 extracted using a lowpass then a highpass elliptical filter'
JBs_FB_title = r'Diurnal temperature values from James Bay at hour resolution from June 2015 to Sept 2015 extracted using a bandpass elliptical filter'
JBs_FL.set_title("\n".join(wrap(JBs_FL_title, 90)))
JBs_FH.set_title("\n".join(wrap(JBs_FH_title, 90)))
JBs_FHL.set_title("\n".join(wrap(JBs_FHL_title, 90)))
JBs_FB.set_title("\n".join(wrap(JBs_FB_title, 90)))

JBs_Plot2.set_xlabel(r'Dates')
JBs_FL.set_xlabel(r'Dates')
JBs_FH.set_xlabel(r'Dates')
JBs_FHL.set_xlabel(r'Dates')
JBs_FB.set_xlabel(r'Dates')

JBs_Plot2.set_ylabel(r'Temperature [$^\circ$ C]')
JBs_FL.set_ylabel(r'Temperature [$^\circ$ C]')
JBs_FH.set_ylabel(r'Temperature [$^\circ$ C]')
JBs_FHL.set_ylabel(r'Temperature [$^\circ$ C]')
JBs_FB.set_ylabel(r'Temperature [$^\circ$ C]')

fig.tight_layout()
plt.show()

# Plot for JBw
dr_JBw = pd.date_range(JBw.index[0], JBw.index[-1], freq='m')
t_JBw = JBw.index.values

fig, (JBw_Plot2, JBw_FH, JBw_FL, JBw_FHL, JBw_FB) = plt.subplots(5, 1, figsize=(10, 16))
JBw_Plot2.plot(JBw, linewidth=1)
JBw_FL.plot(t_JBw[:len(JBwFH)], JBwFH)
JBw_FH.plot(t_JBw[:len(JBwFL)], JBwFL)
JBw_FHL.plot(t_JBw[:len(JBwFHL)], JBwFHL)
JBw_FB.plot(t_JBw[:len(JBwFB)], JBwFB)

JBw_Plot2.xaxis.set_major_locator(mp.dates.MonthLocator(bymonthday = (1)))
JBw_FL.xaxis.set_major_locator(mp.dates.MonthLocator(bymonthday = (1)))
JBw_FH.xaxis.set_major_locator(mp.dates.MonthLocator(bymonthday = (1)))
JBw_FHL.xaxis.set_major_locator(mp.dates.MonthLocator(bymonthday = (1)))
JBw_FB.xaxis.set_major_locator(mp.dates.MonthLocator(bymonthday = (1)))

JBw_Plot2.xaxis.set_major_formatter(DateFormatter('%b-%Y'))
JBw_FL.xaxis.set_major_formatter(DateFormatter('%b-%Y'))
JBw_FH.xaxis.set_major_formatter(DateFormatter('%b-%Y'))
JBw_FHL.xaxis.set_major_formatter(DateFormatter('%b-%Y'))
JBw_FB.xaxis.set_major_formatter(DateFormatter('%b-%Y'))

JBw_Plot2.set_title(r'Temperature values from James Bay at hour resolution from Nov 2014 to Feb 2015')

JBw_FL_title = r'Diurnal temperature values from James Bay at hour resolution from Nov 2014 to Feb 2015 extracted using a lowpass elliptical filter'
JBw_FH_title = r'Diurnal temperature values from James Bay at hour resolution from Nov 2014 to Feb 2015 extracted using a highpass elliptical filter'
JBw_FHL_title = r'Diurnal temperature values from James Bay at hour resolution from Nov 2014 to Feb 2015 extracted using a lowpass then a highpass elliptical filter'
JBw_FB_title = r'Diurnal temperature values from James Bay at hour resolution from Nov 2014 to Feb 2015 extracted using a bandpass elliptical filter'
JBw_FL.set_title("\n".join(wrap(JBw_FL_title, 90)))
JBw_FH.set_title("\n".join(wrap(JBw_FH_title, 90)))
JBw_FHL.set_title("\n".join(wrap(JBw_FHL_title, 90)))
JBw_FB.set_title("\n".join(wrap(JBw_FB_title, 90)))

JBw_Plot2.set_xlabel(r'Dates')
JBw_FL.set_xlabel(r'Dates')
JBw_FH.set_xlabel(r'Dates')
JBw_FHL.set_xlabel(r'Dates')
JBw_FB.set_xlabel(r'Dates')

JBw_Plot2.set_ylabel(r'Temperature [$^\circ$ C]')
JBw_FL.set_ylabel(r'Temperature [$^\circ$ C]')
JBw_FH.set_ylabel(r'Temperature [$^\circ$ C]')
JBw_FHL.set_ylabel(r'Temperature [$^\circ$ C]')
JBw_FB.set_ylabel(r'Temperature [$^\circ$ C]')

fig.tight_layout()
plt.show()


<Figure size 720x1152 with 5 Axes>

<Figure size 720x1152 with 5 Axes>

<Figure size 720x1152 with 5 Axes>

<Figure size 720x1152 with 5 Axes>

## Question 2: Spatial interpolation 

### a) Discussion of data gridding
Data gridding is the process that produces a grid of regularly spaced data from randomly spaced data. Missing values are generated by interpolating the present values. Interpolation assigns a weight to each peasent value, usually based on the distance from the missing point to the value, then uses the weighted values to generate a value for the missing data. This processs is repeated for all missing values in the grid. 

### b) Discussion of three fundamental spatial interpolation schemes 
**Nearest neighbour**: As the name suggests, this interpolation scheme uses the value of the nearest neighbour to interpolate the missing value. Benefits of this scheme is that it is incredibly fast and requires minimal computation. However, accuracy of the interpolation decreases with distance from the nearest neighbouring value, thus it should only be used when interpolating values close to the sample location, or in situations where sample values are in close proximity to each other. It should not be used on sample data which are very spread out. 

**Inverse square distance**: 
Inverse square distance interpalation assigns weights to existing sampled values porportional to the inverse distance to the location of the interpolation. This method is quite fast but it can produce circular spots around the sampled values if a high weight is used (the coefficient for the inverse square distance). It is best used where data is moderatly spaced, and should not be used for data with outliers as it will assign a greater weight to interpolation locations close to the outlier. 

**Exponential decay**:
The expontial decay interpolation scheme decreases the weight of the sampled values exponentially based on their distance to the interpolation loctaion. This methord of interpolation produces results that is the most representative of the original data out of the three interpolation schemes, but it is the most computationally intensive. It should be used for data where sample values are spaced out, but should not be used for interplolating data with closely spaced sample values as more efficient methods can be utilized (such as the nearest neighbour). 

## Question 3: Interpolation

In [15]:
fig, (US_Plot, USm_Plot) = plt.subplots(2, 1, figsize=(12, 8))

US_Plot.plot(US, linewidth=1)
US_Plot.xaxis.set_major_locator(mp.dates.MonthLocator(bymonthday = (1)))
US_Plot.xaxis.set_major_formatter(DateFormatter('%b-%Y'))
US_Plot.set_title("Hour resolution measuerments of temperature at UVicSCI from 2015-01-01 to 2015-12-31")
US_Plot.set_xlabel("Time")
US_Plot.set_ylabel(r"Temperature [$^\circ$ C]")

USm_Plot.plot(USm, linewidth=1)
USm_Plot.xaxis.set_major_formatter(DateFormatter('%d-%b'))
USm_Plot.set_title("Hour resolution measuerments of temperature at UVicSCI from 2015-05-01 to 2015-05-31")
USm_Plot.set_xlabel("Time")
USm_Plot.set_ylabel(r"Temperature [$^\circ$ C]")
# plt.xticks(rotation=90)
fig.tight_layout()
plt.show()

<Figure size 864x576 with 2 Axes>

In [16]:
# Create an function the creates the interpolated data and plots
# ==============================================================
def interp_plot(Array=AS, ti='2016-05-01 00:00', tf='2016-06-01 00:00', 
                Ti='2016-01-01 00:00' , Tf='2016-12-30 23:59', fs=1/3600, 
                Tsub=0.6, rate=3600, nseg=2, tick_frequency='D',  date_format='%d-%b'):
    
    T = (24/Tsub * 3600)/rate

    # Find the power spectra
    # ======================
    # Generate spaces
    Data = Array.loc[Ti:Tf]
    Data_H = [Data[i] for i in range(len(Data)) if int(i%(24/Tsub)) == 0]
    x = np.linspace(0, len(Data), len(Data))
    x_H = [i for i in range(len(Data)) if int(i%(24/Tsub)) == 0]
    x_new = np.linspace(0, x_H[-1], int(len(x_H)*T))
    # Do the interpolation
    Data_lin = interp1d(x_H, Data_H, kind='linear')(x_new)
    Data_cub = interp1d(x_H, Data_H, kind='cubic')(x_new)
    # De-Nan
    Data_DN = Data[~np.isnan(Data)]
    Data_HDN = Data_H
    Data_lin_DN = Data_lin[~np.isnan(Data_lin)]
    Data_cub_DN = Data_cub[~np.isnan(Data_cub)]
    F = 1/rate
    
    # Error rate
    Terr = (np.sum((Data-Data_cub[0:len(Data)])**2))**0.5 / len(Data)

    # Lengths 
    [Data_lin_DN_L, Data_cub_DN_L, Data_DN_L, Data_HDN_L] = np.array([len(Data_lin_DN), len(Data_cub_DN), len(Data_DN), len(Data_HDN)])/(2**nseg)

    # Now calculate
    FDat, GDat = welch(Data_DN, fs, nperseg=Data_DN_L,\
                window=sci.signal.windows.hann(int(Data_DN_L)),\
                noverlap = Data_DN_L/2, nfft = Data_DN_L, detrend=False,\
                return_onesided=True, scaling = 'spectrum')
                        
    FDah, GDah = welch(Data_HDN, 1/(24/Tsub * rate), nperseg=Data_HDN_L,\
                window=sci.signal.windows.hann(int(Data_HDN_L)),\
                noverlap = Data_HDN_L/2, nfft = Data_HDN_L, detrend=False,\
                return_onesided=True, scaling = 'spectrum')
    
    FLin, GLin = welch(Data_lin_DN, F, nperseg=Data_lin_DN_L,\
                window=sci.signal.windows.hann(int(Data_lin_DN_L)),\
                noverlap = Data_lin_DN_L/2, nfft = Data_lin_DN_L, detrend=False,\
                return_onesided=True, scaling = 'spectrum')

    FCub, GCub = welch(Data_cub_DN, F, nperseg=Data_cub_DN_L,\
                window=sci.signal.windows.hann(int(Data_cub_DN_L)),\
                noverlap = Data_cub_DN_L/2, nfft = Data_cub_DN_L, detrend=False,\
                return_onesided=True, scaling = 'spectrum')
    
    # Plot the data
    # =============
    fig, (IntPlt, IntPS) = plt.subplots(2, 1, figsize=(10,8))
    
    IntPlt.scatter(x_H, Data_H, linewidth=0, s=8, c='k', zorder=5, label='Sampled data')
    IntPlt.plot(x, Data, alpha=0.7, zorder=0, label='Original data')
    IntPlt.plot(x_new, Data_lin, alpha=1, zorder=1, label='Linear interpolation')
    IntPlt.plot(x_new, Data_cub, alpha=1, zorder=2, label='Cubic spline interpolation. Error: $T_e ={0:.4f}$'.format(Terr))
    
    t_range = pd.date_range(Data.index[0], Data.index[-1], freq='D')
    xformat = [i.strftime(date_format) for i in t_range]
    
    IntPlt.set_xticklabels(['03-May', '03-May', xformat[127], xformat[131], xformat[135], xformat[139], xformat[143], xformat[147], xformat[150]]) # <======
    
    IntPlt.set_xlabel(r'Date')
    IntPlt.set_ylabel(r'Temperature [$^\circ$ C]')
    IntTitle = r'Hour resolution temperature values from UvicSCI from {0} to {1} with {2:.2f} day sampling rate interpolations'.format(ti, tf, Tsub)
    IntPlt.set_title("\n".join(wrap(IntTitle, 90)))

    IntPlt.set_xlim(Data.index.get_loc(pd.Timestamp(ti)), Data.index.get_loc(pd.Timestamp(tf)))
    IntPlt.legend()
    
    IntPS.loglog(FDat/(2*pi), GDat, zorder=1, label='Original data')
    IntPS.loglog(FLin/(2*pi), GLin, zorder=2, label='Linear interpolation')
    IntPS.loglog(FCub/(2*pi), GCub, zorder=3, label='Cubic spline interpolation')
    IntPS.loglog(FDah/(2*pi), GDah, zorder=4, label='Sampled data')
    IntPSTitle = 'Power spectra of hour resolution temperature values from UvicSCI from {0} to {1} with {2:.2f} day sampling rate interpolations'.format(Ti, Tf, Tsub)
    IntPS.set_title("\n".join(wrap(IntPSTitle, 90)))
    IntPS.set_ylabel(r'$G_{xx}(f)$ [$\frac{V^2}{Hz}$]')
    IntPS.set_xlabel('Frequency [Hz]')
    IntPS.legend()
    
    fig.tight_layout()
    plt.show()

In [17]:
# Plot the graph for 0.6 day sampling rate
interp_plot()

<Figure size 720x576 with 2 Axes>

In [18]:
# Plot the graph for 0.45 day sampling rate
interp_plot(Tsub=0.45)

<Figure size 720x576 with 2 Axes>

### Discussing the spectra/spectrum
Spectral functions appear to closely reperesent each other at low frequencies, more perciesely, until the spectral function for the sampled data ends, then the spectral functions for the interpolated values diverges from that of the original data. This is possibly due to the generation of interpolated values generating additional frequencies not present in the original data. 

### What increasing the sampling rate entail
The root mean square error decreased from $T_e = 0.0317$ to $T_e = 0.0325$ by increasing the sampling rate from 0.45 to 0.6 day rate. This implies the error will approch zero as the sampling rate increases (up to the sample number of the original data), which indicates the ability of the inerpolator to recover the original data increases with the sample rate. Minimal value for the root mean square error for both sampling rates indicates the interpolation method (cubic) is an appropriate method for interpolating data.
